In [8]:
import pandas as pd

In [9]:
# Read the CSV file into a DataFrame
df = pd.read_csv("OxCGRT_simplified_v1.csv")

/var/folders/4g/vvgynjbd1113_p3wdnd530280000gn/T/ipykernel_65534/201225288.py:2: DtypeWarning: Columns (2,3,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("OxCGRT_simplified_v1.csv")


In [10]:
df.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1M_combined_numeric', 'C1M_combined',
       'C2M_combined_numeric', 'C2M_combined', 'C3M_combined_numeric',
       'C3M_combined', 'C4M_combined_numeric', 'C4M_combined',
       'C5M_combined_numeric', 'C5M_combined', 'C6M_combined_numeric',
       'C6M_combined', 'C7M_combined_numeric', 'C7M_combined',
       'C8EV_combined_numeric', 'C8EV_combined', 'E1_combined_numeric',
       'E1_combined', 'E2_combined_numeric', 'E2_combined',
       'H1_combined_numeric', 'H1_combined', 'H2_combined_numeric',
       'H2_combined', 'H3_combined_numeric', 'H3_combined',
       'H6M_combined_numeric', 'H6M_combined', 'H7_combined_numeric',
       'H7_combined', 'H8M_combined_numeric', 'H8M_combined', 'V1..summary.',
       'V2..summary.', 'V3..summary.', 'V4..summary.', 'ConfirmedCases',
       'ConfirmedDeaths', 'MajorityVaccinated', 'PopulationVaccinated',
       'StringencyIndex_Average', 'Governme

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390909 entries, 0 to 390908
Data columns (total 50 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   CountryName                      390909 non-null  object 
 1   CountryCode                      390909 non-null  object 
 2   RegionName                       188149 non-null  object 
 3   RegionCode                       188149 non-null  object 
 4   Jurisdiction                     390909 non-null  object 
 5   Date                             390909 non-null  int64  
 6   C1M_combined_numeric             390909 non-null  float64
 7   C1M_combined                     390909 non-null  object 
 8   C2M_combined_numeric             390909 non-null  float64
 9   C2M_combined                     390909 non-null  object 
 10  C3M_combined_numeric             390909 non-null  float64
 11  C3M_combined                     390909 non-null  object 
 12  C4

In [12]:
df.isnull().sum()

CountryName                             0
CountryCode                             0
RegionName                         202760
RegionCode                         202760
Jurisdiction                            0
Date                                    0
C1M_combined_numeric                    0
C1M_combined                            0
C2M_combined_numeric                    0
C2M_combined                            0
C3M_combined_numeric                    0
C3M_combined                            0
C4M_combined_numeric                    0
C4M_combined                            0
C5M_combined_numeric                    0
C5M_combined                            0
C6M_combined_numeric                    0
C6M_combined                            0
C7M_combined_numeric                    0
C7M_combined                            0
C8EV_combined_numeric                   0
C8EV_combined                           0
E1_combined_numeric                     0
E1_combined                       

In [13]:
df.head()

,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1M_combined_numeric,C1M_combined,C2M_combined_numeric,C2M_combined,...,V3..summary.,V4..summary.,ConfirmedCases,ConfirmedDeaths,MajorityVaccinated,PopulationVaccinated,StringencyIndex_Average,GovernmentResponseIndex_Average,ContainmentHealthIndex_Average,EconomicSupportIndex
0,Aruba,ABW,NaN,NaN,NAT_TOTAL,20200101,0.0,0,0.0,0,...,0,NaN,0.0,0.0,NV,0.0,0.0,0.0,0.0,0.0
1,Aruba,ABW,NaN,NaN,NAT_TOTAL,20200102,0.0,0,0.0,0,...,0,NaN,0.0,0.0,NV,0.0,0.0,0.0,0.0,0.0
2,Aruba,ABW,NaN,NaN,NAT_TOTAL,20200103,0.0,0,0.0,0,...,0,NaN,0.0,0.0,NV,0.0,0.0,0.0,0.0,0.0
3,Aruba,ABW,NaN,NaN,NAT_TOTAL,20200104,0.0,0,0.0,0,...,0,NaN,0.0,0.0,NV,0.0,0.0,0.0,0.0,0.0
4,Aruba,ABW,NaN,NaN,NAT_TOTAL,20200105,0.0,0,0.0,0,...,0,NaN,0.0,0.0,NV,0.0,0.0,0.0,0.0,0.0


In [14]:
# Select only the required columns
selected_columns = ['CountryName', 'RegionName', 'Date', 'ConfirmedCases', 'ConfirmedDeaths', 'PopulationVaccinated', 'StringencyIndex_Average', 'EconomicSupportIndex']
df = df[selected_columns]

In [15]:
# Remove rows with any missing values
df = df.dropna()

# Export the cleaned DataFrame to a new CSV file
df.to_csv("cleaned_data.csv", index=False)

print("Data exported successfully.")

Data exported successfully.


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 187634 entries, 7672 to 381044
Data columns (total 8 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   CountryName              187634 non-null  object 
 1   RegionName               187634 non-null  object 
 2   Date                     187634 non-null  int64  
 3   ConfirmedCases           187634 non-null  float64
 4   ConfirmedDeaths          187634 non-null  float64
 5   PopulationVaccinated     187634 non-null  object 
 6   StringencyIndex_Average  187634 non-null  float64
 7   EconomicSupportIndex     187634 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 12.9+ MB


In [17]:
df.isnull().sum()

CountryName                0
RegionName                 0
Date                       0
ConfirmedCases             0
ConfirmedDeaths            0
PopulationVaccinated       0
StringencyIndex_Average    0
EconomicSupportIndex       0
dtype: int64